In [31]:
import pandas as pd
import functools

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [109]:
train_df=pd.read_csv('../data/processed_train.csv')
test_df=pd.read_csv('../data/processed_test.csv')

In [110]:
train_df = train_df.drop(['id','modified_keyword', 'modified_location', 'url_count', 'urls', 'emoji_count'], axis=1)

In [111]:
train_df = train_df.dropna()

In [112]:
train_df.text.isnull().sum()

0

In [114]:
# tf.enable_eager_execution()
# tf.executing_eagerly()


training_df: pd.DataFrame = train_df
# features = ['text']
print(training_df)

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(training_df['text'].values, tf.string),
            tf.cast(training_df['target'].values, tf.int32)
        )
    )
)

# for features_tensor, target_tensor in training_dataset:
#     print(f'features:{features_tensor} target:{target_tensor}')

                                                   text  target
0     Our Deeds are the Reason of this #earthquake M...       1
1                Forest fire near La Ronge Sask. Canada       1
2     All residents asked to 'shelter in place' are ...       1
3     13,000 people receive #wildfires evacuation or...       1
4     Just got sent this photo from Ruby #Alaska as ...       1
...                                                 ...     ...
7608  Two giant cranes holding a bridge collapse int...       1
7609  @aria_ahrary @TheTawniest The out of control w...       1
7610         M1.94 [0104 UTC]?5km S of Volcano Hawaii.        1
7611  Police investigating after an e-bike collided ...       1
7612  The Latest More Homes Razed by Northern Califo...       1

[7584 rows x 2 columns]


In [115]:
training_dataset

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int32)>

In [98]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(2048, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [99]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [100]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(train_df.text)
train_text = tokenizer.texts_to_sequences(train_df.text)
train_text = tf.keras.preprocessing.sequence.pad_sequences(train_text,padding='post')

In [118]:
train_text.shape

(7584, 31)

In [102]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          131072    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 205,441
Trainable params: 205,441
Non-trainable params: 0
_________________________________________________________________


In [117]:
history = model.fit(training_dataset, epochs=1)

ValueError: in user code:

    /usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:789 run_step  **
        outputs = model.t